In [53]:
import csv
import numpy as np
from raster_chunks import GeoChunks as gc

In [12]:
with open('./out.csv','r') as f:
    rdr = csv.reader(f)
    i = 0
    arr = []
    for row in rdr:
        if i >= 1:
            arr.append([float(row[1]),float(row[2]),float(row[0])])
        i = i+1
        

In [13]:
np_arr = np.array(arr)


In [21]:
xm = np.mean(np_arr[:,0])
ym = np.mean(np_arr[:,1])
zm = np.mean(np_arr[:,2])

In [29]:
i_arr = np.dstack((np_arr[:,0] - xm,np_arr[:,1] - ym, np_arr[:,2] - zm))

In [28]:
(np_arr[:,0] - xm).shape

(56,)

In [30]:
i_arr.shape

(1, 56, 3)

In [38]:
i2_arr = i_arr.reshape(np_arr.shape)

In [45]:
?np.linalg.svd

In [46]:
u,s,v = np.linalg.svd(i2_arr)

In [47]:
_v = np.linalg.inv(v)

In [48]:
_v.shape

(3, 3)

In [49]:
_n = np.zeros((3,1))

In [50]:
_n[2] = 1

In [51]:
_out = np.matmul(_v,_n)

In [65]:
ref_tif = gc('/home/madhavm/vimana/storage/rough/cs_grid/2017-04-29_T_UTM_dem.tif')
new_tif = gc.create_from(ref_tif,'./out3.tif')

In [55]:
def z_from_plane(x,y,_ym = ym, _xm = xm, norm = _out):
    z = ((_xm - x)*norm[0] + (_ym - y)*norm[1])/norm[2] + zm
    return z

In [66]:
ref_arr = ref_tif.read()
ref_arr.shape

(3530, 3887)

In [67]:
ref_tif.geo_trans

(804366.6605750369,
 0.025915953291487295,
 0.0,
 1438133.3171791409,
 0.0,
 -0.025915953291487295)

In [68]:
ref_tif.data_gtif.RasterXSize

3887

In [69]:
new_arr = np.zeros(ref_arr.shape)
for x in range(ref_arr.shape[1]):
    for y in range(ref_arr.shape[0]):
        if(ref_arr[y,x] == ref_tif.no_data_value):
            new_arr[y,x] = new_tif.no_data_value
        else:
            #new_arr[y,x] = 1
            xc = ref_tif.geo_trans[0] + x * ref_tif.geo_trans[1]
            yc = ref_tif.geo_trans[3] + y * ref_tif.geo_trans[5]
            new_arr[y,x] = z_from_plane(xc,yc)
new_tif.write(new_arr)
        

0

In [70]:
new_tif = None

In [61]:
ref_tif.no_data_value

-32767.0

In [71]:
import dem_processing.dem_filtering as df


In [72]:
df.volumeRefPlane('./out3.tif')

4774377.048679838